## Import Libraries

In [1]:
import pandas as pd
import numpy as np
import os
import glob

In [2]:
def is_non_zero_file(fpath):  
    if os.path.isfile(fpath):
        tmp = pd.read_csv(fpath)
        if len(tmp.columns)>1:
            return True
    return False

In [3]:
csv_dir = "/home/mahirwar/Desktop/Monika/npsad_data/vivek/reports/New-Minerva-Data-output/yp2mf3i8_epoch=108-step=872.ckpt"

In [4]:
csv_files = glob.glob(os.path.join(csv_dir, "*.csv"))

In [5]:
csv_files_df = [pd.read_csv(csv_file) for csv_file in csv_files if is_non_zero_file(csv_file)]

In [6]:
all_plaque_objects  = pd.concat(csv_files_df, axis=0, ignore_index=True)

In [7]:
all_plaque_objects["XENum"]=all_plaque_objects["image_name"].apply(lambda l: l.split("_1_")[0])

## Compute Brown Pixels

In [15]:
brown_pixel_crop_wise = all_plaque_objects[["image_name","brown_pixels","XENum"]].drop_duplicates()

In [16]:
brown_pixel_crop_wise = brown_pixel_crop_wise.groupby(["XENum"])["brown_pixels"].sum().reset_index()

In [17]:
all_plaque_objects.head(2)

,image_name,label,confidence,brown_pixels,core,coarse_grained,diffuse,caa,centroid,eccentricity,area,equivalent_diameter,mask_present,total_processing_time,processing_time,XENum
0,XE10-032_1_AmyB_1x_7168_y_111616.png,CAA,0.818313,1332,0,0,0,1,"(81.05560418061062, 84.85475982083098)",0.785600,19423,157.258169,1,927.892682,NaN,XE10-032
1,XE10-032_1_AmyB_1x_21504_y_122880.png,Diffuse,0.989099,81,0,0,1,0,"(180.78206284247764, 190.18450440452244)",0.392877,115336,383.210590,1,927.892682,NaN,XE10-032


## Compute plaque-wise area, eccentricity features

In [25]:
area_ecc = all_plaque_objects.groupby(["XENum","label"]).agg({"area":"mean","eccentricity":"mean"}).reset_index()

In [29]:
area_ecc.index = area_ecc["XENum"]

,XENum,label,area,eccentricity
XENum,,,,
XE05-122,XE05-122,CAA,19031.000000,0.681711
XE05-122,XE05-122,Coarse-Grained,19160.250000,0.554927
XE05-122,XE05-122,Diffuse,21024.045455,0.537233
XE07-002,XE07-002,CAA,57619.230769,0.716626
XE07-002,XE07-002,Coarse-Grained,30766.400000,0.589762
...,...,...,...,...
XE19-052,XE19-052,Coarse-Grained,5386.750000,0.499638
XE19-052,XE19-052,Diffuse,8705.250000,0.667550
XE19-054,XE19-054,CAA,2662.000000,0.461951


In [33]:
area_ecc_1 = area_ecc.pivot(index="XENum",columns="label", values=["area","eccentricity"]).reset_index()

In [35]:
area_ecc_1

XENum          area                                              \
label                     CAA Coarse-Grained          Cored       Diffuse   
0      XE05-122  19031.000000   19160.250000            NaN  21024.045455   
1      XE07-002  57619.230769   30766.400000   34568.500000  14275.008428   
2      XE07-027  43831.973684   37887.000000  132505.250000  22779.035211   
3      XE07-033  51516.692308   13041.367647   28770.285714  10450.964567   
4      XE07-034  45353.039474   10368.570652   32486.131313  11088.460952   
..          ...           ...            ...            ...           ...   
293    XE19-049  23016.780488   15138.252050   43346.076923  10068.051013   
294    XE19-050  13959.000000    5019.000000            NaN  31923.750000   
295    XE19-051  29820.200000    3942.800000            NaN   8134.478261   
296    XE19-052  19407.666667    5386.750000            NaN   8705.250000   
297    XE19-054   2662.000000    3826.000000            NaN  29529.000000   

      eccentricity                                     
label          CAA Coarse-Grained     Cored   Diffuse  
0         0.681711       0.554927       NaN  0.537233  
1         0.716626       0.589762  0.482498  0.618842  
2         0.543752       0.500939  0.527288  0.633733  
3         0.564527       0.545150  0.536949  0.594557  
4         0.570463       0.526480  0.523137  0.597037  
..             ...            ...       ...       ...  
293       0.609468       0.538394  0.514562  0.619654  
294       0.742436       0.466612       NaN  0.760454  
295       0.643812       0.573117       NaN  0.600902  
296       0.483418       0.499638       NaN  0.667550  
297       0.461951       0.653398       NaN  0.399501  

[298 rows x 9 columns]

In [36]:
area_ecc_1.columns = ["XENum","area_CAA","area_Coarse-Grained","area_Cored","area_Diffuse", "eccentricity_CAA","eccentricity_Coarse-Grained","eccentricity_Cored","eccentricity_Diffuse"]

In [37]:
area_ecc_1

,XENum,area_CAA,area_Coarse-Grained,area_Cored,area_Diffuse,eccentricity_CAA,eccentricity_Coarse-Grained,eccentricity_Cored,eccentricity_Diffuse
0,XE05-122,19031.000000,19160.250000,NaN,21024.045455,0.681711,0.554927,NaN,0.537233
1,XE07-002,57619.230769,30766.400000,34568.500000,14275.008428,0.716626,0.589762,0.482498,0.618842
2,XE07-027,43831.973684,37887.000000,132505.250000,22779.035211,0.543752,0.500939,0.527288,0.633733
3,XE07-033,51516.692308,13041.367647,28770.285714,10450.964567,0.564527,0.545150,0.536949,0.594557
4,XE07-034,45353.039474,10368.570652,32486.131313,11088.460952,0.570463,0.526480,0.523137,0.597037
...,...,...,...,...,...,...,...,...,...
293,XE19-049,23016.780488,15138.252050,43346.076923,10068.051013,0.609468,0.538394,0.514562,0.619654
294,XE19-050,13959.000000,5019.000000,NaN,31923.750000,0.742436,0.466612,NaN,0.760454
295,XE19-051,29820.200000,3942.800000,NaN,8134.478261,0.643812,0.573117,NaN,0.600902
296,XE19-052,19407.666667,5386.750000,NaN,8705.250000,0.483418,0.499638,NaN,0.667550


## Clinical Metadata

In [2]:
clinical_data = pd.read_csv("/gladstone/finkbeiner/steve/work/data/npsad_data/vivek/Metadata/clinical_metadata_v2_lates.csv")

In [3]:
clinical_data.columns[:20]

Index(['SubID', 'Brain_bank', 'Age', 'Sex', 'Sex_chr_aneuploidy', 'Ethnicity',
       'Dx', 'pH', 'PMI', 'Death_Time', 'Death_Month', 'Death_Season', 'CERAD',
       'BRAAK_AD', 'BRAAK_PD', 'CDRScore', 'Plq_Mn', 'Plq_Mn_MFG', 'ApoE_gt',
       'CMC_individual_ID'],
      dtype='object')

In [12]:
clinical_data.columns[20:40]

Index(['AMPAD_msbb_individualID', 'snRNAseq_ID', 'AD', 'MCI', 'Dementia', 'PD',
       'PD_uncertain_plus_encephalitic', 'DLBD', 'FTD', 'ALS',
       'NormPressHydrocephalus', 'ArgyrophilicGrain', 'CerebralAtrophy',
       'Tauopathy', 'Others_Neurodegenerative', 'MS', 'PSP', 'Epilepsy',
       'Seizures', 'Tumor'],
      dtype='object')

In [4]:
AmyB_MFG_flagged = pd.read_csv("/gladstone/finkbeiner/steve/work/data/npsad_data/vivek/Metadata/overlap_with_metadata_srna_flag/AmyB-MFG_hasClinicalDataflag_hassnRNAflag.csv")

In [5]:
matched_cl = AmyB_MFG_flagged[AmyB_MFG_flagged["hasClinicalData"]==True]

In [28]:
#merge clinical metadata with model prredicted output
#plaque_counts_merged = pd.merge(plaque_counts,clinical_data[["Dx","Imaging_XENum","Plq_Mn","Plq_Mn_MFG","MidPlaquesValue","MidPlaquesWCoresValue","CDRScore","prs_AD2"]], left_on="XENum", right_on="Imaging_XENum", how = "left")
plaque_counts_merged = pd.merge(matched_cl[["XENum","SubNum"]],clinical_data[["Imaging_XENum","Plq_Mn_MFG","MidPlaquesValue","MidPlaquesWCoresValue","CDRScore", 'Age', 'Sex','Ethnicity',"ApoE_gt","AD","DLBD","PMI",'MCI', 'Dementia']], left_on="XENum", right_on="Imaging_XENum", how = "left")


In [29]:
plaque_counts_merged

,XENum,SubNum,Imaging_XENum,Plq_Mn_MFG,MidPlaquesValue,MidPlaquesWCoresValue,CDRScore,Age,Sex,Ethnicity,ApoE_gt,AD,DLBD,PMI,MCI,Dementia
0,XE10-039,81015,XE10-039,8.66,5.0,3.0,3.0,90.0,Female,White,34.0,1.0,0.0,18.916667,0.0,1.0
1,XE18-048,808460,XE18-048,0.00,0.0,0.0,0.0,66.0,Male,White,NaN,0.0,0.0,23.166667,0.0,0.0
2,XE09-012,46146,XE09-012,0.00,0.0,0.0,0.0,71.0,Male,White,33.0,0.0,0.0,5.616667,0.0,0.0
3,XE09-041,20701,XE09-041,8.66,3.0,3.0,1.0,68.0,Male,Hispanic,44.0,1.0,0.0,6.616667,0.0,1.0
4,XE12-015,70195,XE12-015,41.02,5.0,5.0,5.0,59.0,Female,White,44.0,1.0,0.0,4.750000,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
245,XE10-050,24530,XE10-050,5.35,3.0,3.0,0.0,96.0,Female,White,33.0,0.0,1.0,3.750000,0.0,0.0
246,XE09-004,79736,XE09-004,4.08,1.0,1.0,3.0,93.0,Male,White,33.0,1.0,1.0,6.750000,0.0,1.0
247,XE05-123,82307,XE05-123,2.48,1.0,1.0,4.0,98.0,Male,White,34.0,1.0,1.0,3.000000,0.0,1.0
248,XE18-038,22767,XE18-038,NaN,0.0,0.0,0.5,94.0,Female,White,NaN,0.0,0.0,9.833333,1.0,0.0


In [30]:
plaque_counts_merged["CDRScore"].isna().sum()

22

In [31]:
plaque_counts_merged.head(2)

,XENum,SubNum,Imaging_XENum,Plq_Mn_MFG,MidPlaquesValue,MidPlaquesWCoresValue,CDRScore,Age,Sex,Ethnicity,ApoE_gt,AD,DLBD,PMI,MCI,Dementia
0,XE10-039,81015,XE10-039,8.66,5.0,3.0,3.0,90.0,Female,White,34.0,1.0,0.0,18.916667,0.0,1.0
1,XE18-048,808460,XE18-048,0.00,0.0,0.0,0.0,66.0,Male,White,NaN,0.0,0.0,23.166667,0.0,0.0


In [32]:
VA_BR = pd.read_excel("/gladstone/finkbeiner/steve/work/data/npsad_data/vivek/Metadata/overlap_with_metadata_srna_flag/VA_BR_Clinical_combo_matched_CMC_AMPAD.xlsx") 

In [33]:
VA_BR.head(5)

,SubID,SubNum,Batch1_Barcode,Batch1_Storage.Type,Batch1_Brain.Region,Batch1_UoM,Batch1_Size,Batch2_Barcode,Batch2_Storage.Type,Batch2_Brain.Region,...,Braak.Stage,CMC_ATACseq,CMC_ChIPSeq,CMC_microarray,CMC_proteomics,CMC_rnaSeq,CMC_SNP,CMC_WGS,AMPAD_msbb_individualID,Barcode_received
0,M16653,16653.0,NaN,NaN,BM-9/10/46,NaN,NaN,NaN,NaN,NaN,...,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,M3284,3284.0,NaN,NaN,BM-9/10/46,NaN,NaN,NaN,NaN,NaN,...,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,AMPAD_MSSM_0000003284,NaN
2,M12846,12846.0,NaN,NaN,BM-9/10/46,NaN,NaN,NaN,NaN,NaN,...,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,AMPAD_MSSM_0000012846,NaN
3,M16747,16747.0,NaN,NaN,BM-9/10/46,NaN,NaN,NaN,NaN,NaN,...,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,AMPAD_MSSM_0000016747,NaN
4,M28611,28611.0,NaN,NaN,BM-9/10/46,NaN,NaN,NaN,NaN,NaN,...,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,AMPAD_MSSM_0000028611,NaN


In [34]:
VA_BR.columns[:40]

Index(['SubID', 'SubNum', 'Batch1_Barcode', 'Batch1_Storage.Type',
       'Batch1_Brain.Region', 'Batch1_UoM', 'Batch1_Size', 'Batch2_Barcode',
       'Batch2_Storage.Type', 'Batch2_Brain.Region', 'Batch2_UoM',
       'Batch2_Size', 'Batch2_Loc.Type', 'Batch2_Loc', 'Batch2_Shell',
       'Batch2_Box', 'Age', 'Sex', 'Race', 'PMI', 'CDRScore', 'Final.Dx',
       'CERAD_1', 'CERAD_2', 'Plq_Mn', 'HippoPlaquesValue',
       'EntorPlaquesValue', 'AmygPlaquesValue', 'HippoPlaquesWCoresValue',
       'EntorPlaquesWCoresValue', 'AmygPlaquesWCoresValue',
       'HippoTanglesValue', 'EntorTanglesValue', 'AmygTanglesValue',
       'HippoLewyValue', 'EntorLewyValue', 'AmygLewyValue', 'MidPlaquesValue',
       'SupPlaquesValue', 'InfPlaquesValue'],
      dtype='object')

In [20]:
VA_BR.columns[40:]

Index(['OcciPlaquesValue', 'MidPlaquesWCoresValue', 'SupPlaquesWCoresValue',
       'InfPlaquesWCoresValue', 'OcciPlaquesWCoresValue', 'MidGliosisValue',
       'SupGliosisValue', 'InfGliosisValue', 'OcciGliosisValue',
       'MidLewyValue', 'SupLewyValue', 'InfLewyValue', 'OcciLweyValue',
       'LewySubValue', 'LewyNucleusValue', 'LewyLocusValue',
       'LewyDorsalVValue', 'MoodDysCurValue', 'MoodDysHxValue',
       'DecIntCurValue', 'DecIntHxValue', 'WtLossCurValue', 'WtLossHxValue',
       'DecAppCurValue', 'DecAppHxValue', 'WtGainCurValue', 'WtGainHxValue',
       'EarlyInsomCurValue', 'EarlyInsomHxValue', 'MidInsomCurValue',
       'MidInsomHxValue', 'LateInsomCurValue', 'LateInsomHxValue',
       'HypersomCurValue', 'HypersomHxValue', 'PsychoAgiCurValue',
       'PsychoAgiHxValue', 'PsychoRetardCurValue', 'PsychoRetardHxValue',
       'FatCurValue', 'FatHxValue', 'WorthCurValue', 'WorthHxValue',
       'DelCurValue', 'DelHxValue', 'RumCurValue', 'RumHxValue',
       'ThoughtDea

In [35]:
unmatched_cl = AmyB_MFG_flagged[AmyB_MFG_flagged["hasClinicalData"]==False]

In [36]:
VA_BR_short = VA_BR[['SubNum', 'MidPlaquesValue', "MidPlaquesWCoresValue",'CDRScore', "PMI",'Age', 'Sex', 'Race']]

In [37]:
new_clinical_matched = pd.merge(unmatched_cl[["XENum",	"SubNum"]],VA_BR_short, on="SubNum", how="left" )

In [43]:
new_clinical_matched.rename({'Race':'Ethnicity'},axis=1,inplace=True)

In [44]:
new_clinical_matched["Plq_Mn_MFG"] = None

In [45]:
new_clinical_matched

,XENum,SubNum,MidPlaquesValue,MidPlaquesWCoresValue,CDRScore,PMI,Age,Sex,Ethnicity,Plq_Mn_MFG
0,XE17-019,370135,3.0,3.0,1.0,1159.0,87.0,Male,White,None
1,XE16-060,34894,3.0,3.0,2.0,295.0,95.0,Male,White,None
2,XE08-003,43750,0.0,0.0,0.0,360.0,81.0,Female,White,None
3,XE07-034,50158,3.0,3.0,2.0,195.0,88.0,Male,Hispanic,None
4,XE19-023,82747,0.0,0.0,0.0,1120.0,62.0,Male,White,None
...,...,...,...,...,...,...,...,...,...,...
69,XE19-046,206781,0.0,0.0,0.5,312.0,75.0,Female,NaN,None
70,XE08-037,3581,3.0,3.0,3.0,565.0,84.0,Male,White,None
71,XE16-033,47833,3.0,3.0,3.0,300.0,90.0,Female,White,None
72,XE13-007,31742,5.0,5.0,4.0,885.0,87.0,Female,White,None


In [46]:
Final_clinical_data = pd.concat([plaque_counts_merged,new_clinical_matched],axis=0)

In [47]:
Final_clinical_data

,XENum,SubNum,Imaging_XENum,Plq_Mn_MFG,MidPlaquesValue,MidPlaquesWCoresValue,CDRScore,Age,Sex,Ethnicity,ApoE_gt,AD,DLBD,PMI,MCI,Dementia
0,XE10-039,81015,XE10-039,8.66,5.0,3.0,3.0,90.0,Female,White,34.0,1.0,0.0,18.916667,0.0,1.0
1,XE18-048,808460,XE18-048,0.00,0.0,0.0,0.0,66.0,Male,White,NaN,0.0,0.0,23.166667,0.0,0.0
2,XE09-012,46146,XE09-012,0.00,0.0,0.0,0.0,71.0,Male,White,33.0,0.0,0.0,5.616667,0.0,0.0
3,XE09-041,20701,XE09-041,8.66,3.0,3.0,1.0,68.0,Male,Hispanic,44.0,1.0,0.0,6.616667,0.0,1.0
4,XE12-015,70195,XE12-015,41.02,5.0,5.0,5.0,59.0,Female,White,44.0,1.0,0.0,4.750000,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69,XE19-046,206781,NaN,NaN,0.0,0.0,0.5,75.0,Female,NaN,NaN,NaN,NaN,312.000000,NaN,NaN
70,XE08-037,3581,NaN,NaN,3.0,3.0,3.0,84.0,Male,White,NaN,NaN,NaN,565.000000,NaN,NaN
71,XE16-033,47833,NaN,NaN,3.0,3.0,3.0,90.0,Female,White,NaN,NaN,NaN,300.000000,NaN,NaN
72,XE13-007,31742,NaN,NaN,5.0,5.0,4.0,87.0,Female,White,NaN,NaN,NaN,885.000000,NaN,NaN


In [50]:
Final_clinical_data.to_csv("/gladstone/finkbeiner/steve/work/data/npsad_data/vivek/reports/Clinical_correlation_analysis/Mapped_clinical_data_full_minerava.csv")

In [51]:
Final_clinical_data.columns

Index(['XENum', 'SubNum', 'Imaging_XENum', 'Plq_Mn_MFG', 'MidPlaquesValue',
       'MidPlaquesWCoresValue', 'CDRScore', 'Age', 'Sex', 'Ethnicity',
       'ApoE_gt', 'AD', 'DLBD', 'PMI', 'MCI', 'Dementia'],
      dtype='object')

In [53]:
Final_clinical_data1 = Final_clinical_data.drop(["Imaging_XENum", 'Age', 'Sex', 'Ethnicity'],axis=1)

In [54]:
plaque_counts = pd.read_csv("/gladstone/finkbeiner/steve/work/data/npsad_data/vivek/csv_merged_data/Full_minerva_output.csv")


In [55]:
plaque_counts["XENum"]=plaque_counts["image_name"].apply(lambda l: l.split("_1_")[0])

In [57]:
plaque_counts_merged = pd.merge(plaque_counts, Final_clinical_data1, on="XENum", how="left")

In [58]:
plaque_counts_merged["total_plaques"] = plaque_counts_merged["total_core_plaques"]+plaque_counts_merged["total_coarse_grained_plaques"]+plaque_counts_merged["total_diffuse_plaques"]+plaque_counts_merged["total_caa_plaques"]

In [59]:
plaque_counts_merged["Dementia_group"] = np.where(plaque_counts_merged["CDRScore"]<=0, "Not Dementia", np.where(plaque_counts_merged["CDRScore"]<=0.5,"MCI", "Dementia"))

In [60]:
plaque_counts_merged["ApoE_e4"] = np.where(plaque_counts_merged["ApoE_gt"].isin([34,44,24]), "ApoE_e4",np.where(plaque_counts_merged["ApoE_gt"]>0, "non APOE_e4",None ))

In [61]:
#plaque_counts_merged.to_csv("/gladstone/finkbeiner/steve/work/data/npsad_data/vivek/reports/Clinical_correlation_analysis/Full_minerva_output_w_Clinical.csv")

## Merge other datasets

In [38]:
plaque_counts_merged = pd.read_csv("/gladstone/finkbeiner/steve/work/data/npsad_data/vivek/reports/Clinical_correlation_analysis/Full_minerva_output_w_Clinical.csv")

In [41]:
plaque_counts_merged = pd.merge(plaque_counts_merged,area_ecc_1, how="left", on="XENum")

In [42]:
plaque_counts_merged = pd.merge(plaque_counts_merged,brown_pixel_crop_wise, how="left", on="XENum")

In [44]:
plaque_counts_merged.columns

Index(['Unnamed: 0', 'image_name', 'total_core_plaques',
       'total_coarse_grained_plaques', 'total_diffuse_plaques',
       'total_caa_plaques', 'avg_area', 'avg_diameter', 'avg_eccentricity',
       'age', 'sex', 'ethinicity', 'Dx', 'XENum', 'SubNum', 'Plq_Mn_MFG',
       'MidPlaquesValue', 'MidPlaquesWCoresValue', 'CDRScore', 'ApoE_gt', 'AD',
       'DLBD', 'PMI', 'MCI', 'Dementia', 'total_plaques', 'Dementia_group',
       'ApoE_e4', 'area_CAA', 'area_Coarse-Grained', 'area_Cored',
       'area_Diffuse', 'eccentricity_CAA', 'eccentricity_Coarse-Grained',
       'eccentricity_Cored', 'eccentricity_Diffuse', 'brown_pixels'],
      dtype='object')

In [45]:
plaque_counts_merged.to_csv("/gladstone/finkbeiner/steve/work/data/npsad_data/vivek/reports/Clinical_correlation_analysis/Full_minerva_output_w_Clinical.csv")